# Built-in Neural Networks in Φ<sub>Flow</sub>

Apart from being a general purpose differential physics library, Φ<sub>Flow</sub> also provides a number of backend-agnostic way of setting up optimizers and some neural networks.

The following network architectures are supported:

* Fully-connected networks: `dense_net()`
* Convolutional networks: `conv_net()`
* Residual networks: `res_net()`
* U-Nets: `u_net()`
* Convolutional classifiers: `conv_classifier()`

In addition to zero-padding, the convolutional neural networks all support circular periodic padding across feature map spatial dimensions to maintain periodicity.

All network-related convenience functions in Φ<sub>Flow</sub> support PyTorch, TensorFlow and Jax/Stax, setting up native networks.


In [ ]:
from phi.tf.flow import *
from phi.jax.stax.flow import *
from phi.torch.flow import *

## Fully-connected Networks
Fully-connected neural networks are available in Φ<sub>Flow</sub> via `dense_net()`.

#### Arguments

* `in_channels` : size of input layer, int
* `out_channels` = size of output layer, int
* `layers` : tuple of linear layers between input and output neurons, list or tuple
* `activation` : activation function used within the layers, string
* `batch_norm` : use of batch norm after each linear layer, bool

In [3]:
net = dense_net(in_channels=1, out_channels=1, layers=[8, 8], activation='ReLU', batch_norm=False)  # Implemented for PyTorch, TensorFlow, Jax-Stax)

In [4]:
optimizer = adam(net, 1e-3)
BATCH = batch(batch=100)

def loss_function(data: Tensor):
    prediction = math.native_call(net, data)
    label = math.sin(data)
    return math.l2_loss(prediction - label), data, label

print(f"Initial loss: {loss_function(math.random_normal(BATCH))[0]}")
for i in range(100):
    loss, _data, _label = update_weights(net, optimizer, loss_function, data=math.random_normal(BATCH))
print(f"Final loss: {loss}")

Initial loss: (batchᵇ=100) 0.278 ± 0.308 (4e-05...1e+00)
Final loss: (batchᵇ=100) 0.095 ± 0.131 (1e-05...9e-01)


## U-Nets
Φ<sub>Flow</sub> provides a built in U-net architecture, classically popular for Semantic Segmentation in Computer Vision, composed of downsampling and upsampling layers.

 #### Arguments

* `in_channels`: input channels of the feature map, dtype : int
* `out_channels` : output channels of the feature map, dtype : int
* `levels` : number of levels of down-sampling and upsampling, dtype : int
* `filters` : filter sizes at each down/up sampling convolutional layer, if the input is integer all conv layers have the same filter size,<br> dtype : int or tuple
* `activation` : activation function used within the layers, dtype : string
* `batch_norm` : use of batchnorm after each conv layer, dtype : bool
* `in_spatial` : spatial dimensions of the input feature map, dtype : int
* `use_res_blocks` : use convolutional blocks with skip connections instead of regular convolutional blocks, dtype : bool

In [5]:
net = u_net(in_channels= 1, out_channels= 2, levels=4, filters=16, batch_norm=True, activation='ReLU', in_spatial=2, use_res_blocks=False)

In [6]:
#Loss function for training in the network to identify noise parameters
def loss_function(scale: Tensor, smoothness: Tensor):
    grid = CenteredGrid(Noise(scale=scale, smoothness=smoothness), x=64, y=64)

    print(f'Grid Shape : {grid.shape}')
    pred_scale, pred_smoothness = field.native_call(net, grid).vector
    return math.l2_loss(pred_scale - scale) + math.l2_loss(pred_smoothness - smoothness)

In [7]:
optimizer = adam(net, learning_rate=1e-3)
gt_scale = math.random_uniform(batch(examples=50), low=1, high=10)
gt_smoothness = math.random_uniform(batch(examples=50), low=.5, high=3)

print(f"Initial loss: {loss_function(gt_scale, gt_smoothness)}")
for i in range(10):
    loss = update_weights(net, optimizer, loss_function, gt_scale, gt_smoothness)
    print(f'Iter : {i}, Loss : {loss}')
print(f"Final loss: {loss}")

Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Initial loss: (examplesᵇ=50) 9.06e+04 ± 6.6e+04 (9e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 0, Loss : (examplesᵇ=50) 9.07e+04 ± 6.6e+04 (9e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 1, Loss : (examplesᵇ=50) 8.68e+04 ± 6.0e+04 (2e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 2, Loss : (examplesᵇ=50) 8.48e+04 ± 5.6e+04 (2e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 3, Loss : (examplesᵇ=50) 8.33e+04 ± 5.4e+04 (2e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 4, Loss : (examplesᵇ=50) 8.22e+04 ± 5.3e+04 (2e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 5, Loss : (examplesᵇ=50) 8.10e+04 ± 5.1e+04 (2e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 6, Loss : (examplesᵇ=50) 8.00e+04 ± 5.0e+04 (2e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 7, Loss : (examplesᵇ=50) 7.90e+04 ± 4.9e+04 (1e+04...2e+05)
Grid Shape : (examples

## Convolutional Networks
Built in Conv-Nets are also provided. Contrary to the classical convolutional neural networks, the feature map spatial size remains the same throughout the layers.
Each layer of the network is essentially a convolutional block comprising of two conv layers.
A filter size of 3 is used in the convolutional layers.

#### Arguments

* `in_channels` : input channels of the feature map, dtype : int
* `out_channels` : output channels of the feature map, dtype : int <br>
* `layers` : list or tuple of output channels for each intermediate layer between the input and final output channels, dtype : list or tuple <br>
* `activation` : activation function used within the layers, dtype : string <br>
* `batch_norm` : use of batchnorm after each conv layer, dtype : bool <br>
* `in_spatial` : spatial dimensions of the input feature map, dtype : int <br>


In [8]:
net = conv_net(in_channels=1, out_channels=2, layers=[2,4,4,2], activation='ReLU', batch_norm=True, in_spatial=2)

In [9]:
optimizer = adam(net, learning_rate=1e-3)
gt_scale = math.random_uniform(batch(examples=50), low=1, high=10)
gt_smoothness = math.random_uniform(batch(examples=50), low=.5, high=3)

print(f"Initial loss: {loss_function(gt_scale, gt_smoothness)}")
for i in range(10):
    loss = update_weights(net, optimizer, loss_function, gt_scale, gt_smoothness)
    print(f'Iter : {i}, Loss : {loss}')
print(f"Final loss: {loss}")

Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Initial loss: (examplesᵇ=50) 9.76e+04 ± 5.7e+04 (8e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 0, Loss : (examplesᵇ=50) 9.77e+04 ± 5.7e+04 (8e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 1, Loss : (examplesᵇ=50) 9.73e+04 ± 5.7e+04 (8e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 2, Loss : (examplesᵇ=50) 9.71e+04 ± 5.7e+04 (8e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 3, Loss : (examplesᵇ=50) 9.68e+04 ± 5.7e+04 (8e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 4, Loss : (examplesᵇ=50) 9.65e+04 ± 5.7e+04 (8e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 5, Loss : (examplesᵇ=50) 9.61e+04 ± 5.7e+04 (8e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 6, Loss : (examplesᵇ=50) 9.58e+04 ± 5.7e+04 (8e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 7, Loss : (examplesᵇ=50) 9.55e+04 ± 5.6e+04 (8e+03...2e+05)
Grid Shape : (examples

## Residual Networks
Built in Res-Nets are provided in the Φ<sub>Flow</sub> framework. Similar to the conv-net, the feature map spatial size remains the same throughout the layers.<br>These networks use residual blocks composed of two conv layers with a skip connection added from the input to the output feature map.<br> A default filter size of 3 is used in the convolutional layers.<br><br>

#### Arguments

* `in_channels` : input channels of the feature map, dtype : int
* `out_channels` : output channels of the feature map, dtype : int
* `layers` : list or tuple of output channels for each intermediate layer between the input and final output channels, dtype : list or tuple
* `activation` : activation function used within the layers, dtype : string
* `batch_norm` : use of batchnorm after each conv layer, dtype : bool
* `in_spatial` : spatial dimensions of the input feature map, dtype : int

In [10]:
net = res_net(in_channels=1, out_channels=2, layers=[2,4,4,2], activation='ReLU', batch_norm=True, in_spatial=2)

In [11]:
optimizer = adam(net, learning_rate=1e-3)
gt_scale = math.random_uniform(batch(examples=50), low=1, high=10)
gt_smoothness = math.random_uniform(batch(examples=50), low=.5, high=3)

print(f"Initial loss: {loss_function(gt_scale, gt_smoothness)}")
for i in range(10):
    loss = update_weights(net, optimizer, loss_function, gt_scale, gt_smoothness)
    print(f'Iter : {i}, Loss : {loss}')
print(f"Final loss: {loss}")

Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Initial loss: (examplesᵇ=50) 6.09e+04 ± 5.1e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 0, Loss : (examplesᵇ=50) 6.12e+04 ± 5.2e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 1, Loss : (examplesᵇ=50) 5.90e+04 ± 4.8e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 2, Loss : (examplesᵇ=50) 5.79e+04 ± 4.8e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 3, Loss : (examplesᵇ=50) 5.66e+04 ± 4.6e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 4, Loss : (examplesᵇ=50) 5.52e+04 ± 4.4e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 5, Loss : (examplesᵇ=50) 5.41e+04 ± 4.2e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 6, Loss : (examplesᵇ=50) 5.31e+04 ± 4.1e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 7, Loss : (examplesᵇ=50) 5.22e+04 ± 4.0e+04 (1e+04...2e+05)
Grid Shape : (examples